##Contents

>>[Contents](#scrollTo=wYQ_wwOcTk3N)

>>[Calculating the rating of products](#scrollTo=B3UZgYMcKJVb)

>>[Importing packages](#scrollTo=MQc73JU3JEHT)

>>[Importing input files](#scrollTo=KfNEdfkVJKQ4)

>>>[Linking to thegoogle drive](#scrollTo=KfNEdfkVJKQ4)

>>>[Defining input and out put paths](#scrollTo=SGYHvAS6JVbS)

>>>[reading input file](#scrollTo=g6FsNDi0JcTP)

>>[Data processing](#scrollTo=jFm-zP2hJjP5)

>>[Defining the rating](#scrollTo=BrheekPyJttW)

>>[Exporting to an output file](#scrollTo=rMVUjdGDKArh)



## Calculating the rating of products
Here I have tried to calculate th rating based on the customer order history. 
It is not simply a number of a product a customer obatined. Because, consider two customers, A and B, who both ordered a product P. Customer A had ordered 30 products in total and among those orders there are 4 of product P. Customer A also might order other products like Q (12 tiems), R (7 times) and S (7 times) . While customer B only ordered 5 products and he/she also had 4 product of P. It seems the most preferred product for custmer A is product Q, but for customer B, it is product P. The ratio of product P respect to all ordered item for customer  A and customer B are 0.13 and 0.8 respectively. These number may indicate that how much customers A and B need or like product P. Then they have to be rescaled respect to most preferred product of that customer. In this way, I can tell which product is the favourite of a customer. These ratio may represent the rating of product P from cusmtomers A and B.

The rating of jth product  ordered by ith customer ($R^i_j$) has calculated as :
\begin{align}
R^i_j & = 100\times\frac{r^i_j}{r^i_{max}} \\
r^i_{max} & = \max(r^i_j) \\
r^i_j & = \frac{n^i_j}{N^i}\\
j&=1,2,...,N^i
\end{align}
where $n^i_j$ is the quantity of jth product ordered by ith customer. $N^i$ is the total numper of products ordered by ith customer. 

##Importing packages

In [0]:
from google.colab import files
from google.colab import drive
import numpy as np
import pandas as pd
import os,sys

##Importing input files
###Linking to thegoogle drive

In [2]:
#To get access to the files on google drive an authorization code it nessacery which will be given by the linked below
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


###Defining input and out put paths 

In [0]:
input_data_path="/gdrive/My Drive/Colab Notebooks/Input_files/produced/"
output_data_path="/gdrive/My Drive/Colab Notebooks/Output_files"

### reading input file

In [4]:
df=pd.read_csv(os.path.join(input_data_path,'data_InstaCart.csv'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33894106 entries, 0 to 33894105
Data columns (total 12 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
product_id                float64
add_to_cart_order         float64
reordered                 float64
aisle_id                  float64
department_id             float64
dtypes: float64(6), int64(5), object(1)
memory usage: 3.0+ GB


##Data processing

Removing the missing data and unnecessary data

In [0]:
df=df.loc[df.eval_set!='test',['user_id','product_id']]

In [6]:
df.head()

,user_id,product_id
0,1,196.0
1,1,12427.0
2,1,14084.0
3,1,26088.0
4,1,26405.0


## Defining the rating

Calculating the number of all products a customer ordered

In [7]:
data=pd.DataFrame({'user_portion':df.groupby(['user_id','product_id']).size()}).reset_index()
data.head()

,user_id,product_id,user_portion
0,1,196.0,11
1,1,10258.0,10
2,1,10326.0,1
3,1,12427.0,10
4,1,13032.0,4


The number of each product a customer has ordered

In [8]:
data=data.join(pd.DataFrame({'product_quantity':df.groupby('user_id').size()}).reset_index().set_index('user_id'),on=('user_id'))
data.head()

,user_id,product_id,user_portion,product_quantity
0,1,196.0,11,70
1,1,10258.0,10,70
2,1,10326.0,1,70
3,1,12427.0,10,70
4,1,13032.0,4,70


The ratio of every product respect to all products a customer has ordered

In [0]:
del df
data.loc[:,'rating']=data.user_portion/data.product_quantity

Tha maximum ratio of products of a customer 

In [0]:
data=data.join(pd.DataFrame({'max_rating':data.groupby('user_id').rating.max()}).reset_index().set_index('user_id'),on=('user_id'))


In [11]:
data.head()

,user_id,product_id,user_portion,product_quantity,rating,max_rating
0,1,196.0,11,70,0.157143,0.157143
1,1,10258.0,10,70,0.142857,0.157143
2,1,10326.0,1,70,0.014286,0.157143
3,1,12427.0,10,70,0.142857,0.157143
4,1,13032.0,4,70,0.057143,0.157143


calculating the rating of a product for a customer. 

In [0]:
data.rating=(data.rating*100/data.max_rating).round()

In [13]:
data.head()

,user_id,product_id,user_portion,product_quantity,rating,max_rating
0,1,196.0,11,70,100.0,0.157143
1,1,10258.0,10,70,91.0,0.157143
2,1,10326.0,1,70,9.0,0.157143
3,1,12427.0,10,70,91.0,0.157143
4,1,13032.0,4,70,36.0,0.157143


## Exporting to an output file

In [0]:
data=data.loc[:,['user_id','product_id','rating']]

In [15]:
data.head()

,user_id,product_id,rating
0,1,196.0,100.0
1,1,10258.0,91.0
2,1,10326.0,9.0
3,1,12427.0,91.0
4,1,13032.0,36.0


In [0]:
data.to_csv(os.path.join(input_data_path,'data_InstaCart_rating.csv'), encoding='utf-8',index=False)